In [1]:
%load_ext autoreload
%autoreload 2

In [216]:
import json
import os
import pandas as pd
from datetime import datetime
import geopandas as gpd
from pyproj import Proj, transform
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from shapely.geometry import Point, Polygon, MultiPolygon, shape
from geopandas.tools import sjoin

#from src.hydra import mongo_utils
from src.hydra import gather_tweets
from src.func import geo_utils
from src.func import tweet_utils
from src.func import regex

In [230]:
tweet_dir = Path("../data/raw/tweets").resolve()
tweet_file = Path("CA_Los_Angeles_0644000.json")
tweet_file.stem[-7:]

'0644000'

In [4]:
# Load a cities worth of tweets
city = "CA_Los_Angeles_0644000.json"
with open("../data/processed/city_tweets/{}.json".format(city),'r') as f:
    tweets = json.load(f)

print("Total tweets in {}: {}".format(city,len(tweets)))


Total tweets in CA_Los_Angeles_0644000: 3597386


In [218]:
tweet_dir = Path("../../data/raw/tweets").resolve()
tweet_dir

PosixPath('/Users/aaron/code/research/data/raw/tweets')

In [212]:
tweets = pd.read_json("../data/raw/tweets/{}.json".format(city))

In [232]:
len(tweets)

3597386

In [190]:
def reproj_coords(coords):
    tweet_crs = Proj({'proj': 'longlat', 'ellps': 'WGS84', 'datum': 'WGS84'})
    tpl_crs = Proj({'proj': 'aea',
     'lat_1': 20,
     'lat_2': 60,
     'lat_0': 40,
     'lon_0': -96,
     'x_0': 0,
     'y_0': 0,
     'datum': 'NAD83',
     'units': 'm',
     'no_defs': True,
     'wktext': True})
    lon = coords[0]
    lat = coords[1]
    #print(lon, lat)
    lon,lat = transform(tweet_crs,tpl_crs,lon,lat)
    return lon, lat

tpl_crs = Proj({'proj': 'aea',
     'lat_1': 20,
     'lat_2': 60,
     'lat_0': 40,
     'lon_0': -96,
     'x_0': 0,
     'y_0': 0,
     'datum': 'NAD83',
     'units': 'm',
     'no_defs': True,
     'wktext': True})

In [87]:
gdf = gpd.read_file('../data/raw/geo/ParkServe_shp_DataShare_05152019/ParkServe_shp_DataShare_05152019/ParkServe_Parks_05152019.shp')
la_parks = gdf[gdf.Park_Pla_1=="0644000"].copy()

# Can save subset of shapes as its own shp file
# la_parks.to_file("CA_Los_Angeles_parks.zip")

In [42]:
#shape(la_parks.iloc[1].geometry)[0].centroid.coords.xy
#p1 = Point(la_parks.iloc[50].geometry.centroid.coords.xy[0][0], la_parks.iloc[50].geometry.centroid.coords.xy[1][0])

(array('d', [-1931597.1650395212]), array('d', [-450151.66028727]))

In [ ]:
## Old way - andys
name_col = "Park_Name"
city_key = "Park_Pla_1"
city_id = "0644000"
pList, nList = geo_utils.loadGeneric('../data/raw/geo/ParkServe_shp_DataShare_05152019/ParkServe_shp_DataShare_05152019/ParkServe_Parks_05152019.shp',
                                     name_col, city_key,city_id)

print("Total park tweets in LA: {}".format(len(park_tweets)))
print("Total control tweets in LA: {}".format(len(control_tweets)))

In [88]:
# Loop way 
la_parks.sort_values(by="Shape_Area", inplace=True)

tweet_sample = tweets[2000000:2000100]
# reproject tweets properly
# multipolyogne issues
# sort tweets
for tweet in tweet_sample:
    point = Point(reproj_tweet(tweet))
    for park in la_parks.iterrows():
        if shape(park[1].geometry).contains(point):
            print(park[1].Park_Name)
            break

El Cariso Regional County Park And Golf Course


In [160]:
# save just a few columns
la_parks_tojoin =la_parks[['geometry','ParkID','Park_Name', 'Shape_Area']].copy()

## Intersection between two geo data frames ##

# Save tweets as df
tweet_df = pd.DataFrame(tweets).set_index('id')
# Drop Language
tweet_df.drop("fastText_lang", axis=1, inplace=True)

# Convert coordinates to points, reprojceted into the TPL projection #
tweet_geometry = [Point(reproj_coords(xy['coordinates'])) for xy in tweet_df.geo.values]

# drop the geo field
tweet_df = tweet_df.drop('geo',axis=1)
# convert to geodataframe, with reprojeted geometry
tweet_gdf = gpd.GeoDataFrame(tweet_df,crs= gdf.crs, geometry=tweet_geometry)
tweet_gdf.head()

In [201]:
# spatial join betweeen tweets and la parks
pointInPolys = sjoin(tweet_gdf, la_parks_tojoin, how='left', op="within")


61783


In [206]:
#pointInPolys.to_file("../la_tweets_parks.geojson", driver="GeoJSON")

In [208]:
park_tweets = pointInPolys.dropna().copy()

In [209]:
park_tweets.to_file("la_parktweets.geojson", driver='GeoJSON')